In [1]:
import json
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import defaultdict
from spellchecker import SpellChecker
from tqdm.notebook import tqdm

nltk.download(['punkt', 'stopwords'], quiet=True);

In [ ]:
filename = 'data/processed_reviews.json'
with open(filename, 'rb') as file:
    reviews = json.load(file)

initial_bag_of_words = defaultdict(set)
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
for review in tqdm(reviews):
    text = review['text'].lower()
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = nltk.word_tokenize(text)
    for word in text:
        if word.isalnum() and word not in stop_words:
            word = lemmatizer.lemmatize(word)
            initial_bag_of_words[review['product_id']].add(word)

counter = defaultdict(int)
for words in initial_bag_of_words.values():
    for word in words:
        counter[word] += 1

keep_words = set()
for word, count in counter.items():
    n = len(initial_bag_of_words)
    if (count < n*.9) or (count > n*.01):
        keep_words.add(word)

reduced_bag_of_words = defaultdict(set)
for product, words in tqdm(initial_bag_of_words.items()):
    for word in words:
        if word in keep_words:
            reduced_bag_of_words[product].add(word)

corrections = {}
spellcheck = SpellChecker()
for word in tqdm(spellcheck.unknown(keep_words)):
    corrections[word] = spellcheck.correction(word)

bag_of_words = defaultdict(set)
for product, words in tqdm(reduced_bag_of_words.items()):
    for word in words:
        if word in corrections:
            bag_of_words[product].add(corrections[word])
        else:
            bag_of_words[product].add(word)

bag_of_words = {product: list(bag_of_words[product]) for product in bag_of_words}

filename = 'data/bag_of_words.json'
with open(filename, 'w') as file:
    json.dump(bag_of_words, file)

In [ ]:
corpus = set()
for words in bag_of_words.values():
    for word in words:
        corpus.add(word)
corpus = list(corpus)

filename = 'data/corpus.json'
with open(filename, 'w') as file:
    json.dump(corpus, file)

In [2]:
filename = 'data/processed_games.json'
with open(filename, 'r') as file:
    games = json.load(file)

bag_of_tags = defaultdict(set)
search_keys = ['publisher', 'genres', 'tags', 'specs', 'developer']
for game in games:
    game_id = game['id']
    for key in search_keys:
        metadata = game[key]
        if isinstance(metadata, str):
            metadata = metadata.lower()
            metadata = re.sub(r'\W', '', metadata)
            bag_of_tags[game_id].add(metadata)
        else:
            metadata = [tag.lower() for tag in metadata]
            metadata = [re.sub(r'\W', '', tag) for tag in metadata]
            bag_of_tags[game_id] = bag_of_tags[game_id].union(metadata)

counter = defaultdict(int)
for words in bag_of_tags.values():
    for word in words:
        counter[word] += 1

keep_words = set()
for word, count in counter.items():
    if count > 1:
        keep_words.add(word)
        
temp = defaultdict(set)
for product, words in bag_of_tags.items():
    for word in words:
        if word in keep_words:
            temp[product].add(word)
bag_of_tags = {product: list(temp[product]) for product in temp}
del temp

filename = 'data/bag_of_tags.json'
with open(filename, 'w') as file:
    json.dump(bag_of_tags, file)

In [5]:
tag_corpus = set()
for words in bag_of_tags.values():
    for word in words:
        tag_corpus.add(word)
tag_corpus = list(tag_corpus)

filename = 'data/tag_corpus.json'
with open(filename, 'w') as file:
    json.dump(tag_corpus, file)

In [5]:
filename = 'data/steam_games.json'
with open(filename, 'r') as file:
    games = json.load(file)

In [6]:
len(games)

32135